In [28]:
from importlib import reload

import ecc
import helper
import script
import tx

In [29]:
from ecc import PrivateKey
from helper import double_sha256, little_endian_to_int
from script import multisig_redeem_script, p2sh_script

secrets = (b'jimmy@programmingblockchain.com test1', b'jimmy@programmingblockchain.com test2')

private_keys = [PrivateKey(little_endian_to_int(double_sha256(s))) for s in secrets]
points = [p.point for p in private_keys]

redeem_script = multisig_redeem_script(2, points)
h160 = redeem_script.hash160()
script_pubkey = p2sh_script(h160)
print(script_pubkey.address(testnet=True))

2MxEZNps15dAnGX5XaVwZWgoDvjvsDE5XSx


In [30]:
from ecc import PrivateKey
from helper import double_sha256, little_endian_to_int
from script import multisig_redeem_script, p2sh_script, Script
from tx import Tx, TxIn, TxOut

secrets = (b'jimmy@programmingblockchain.com test1', b'jimmy@programmingblockchain.com test2')

private_keys = [PrivateKey(little_endian_to_int(double_sha256(s))) for s in secrets]
points = [p.point for p in private_keys]

redeem_script = multisig_redeem_script(2, points)
prev_tx = bytes.fromhex('aefa285e1a83e7153687c478d45fbda4602a6e3a78a3d6412f1e4aac36aef626')
prev_index = 1
fee = int(100000000*0.001)

tx_in = TxIn(prev_tx, prev_index, Script([]), 0xffffffff)
h160 = redeem_script.hash160()
script_pubkey = p2sh_script(h160)
amount = tx_in.value(testnet=True) - fee
tx_out = TxOut(amount, script_pubkey)

t = Tx(1, [tx_in], [tx_out], 0, testnet=True)
t.sign_input_p2sh_multisig(0, private_keys, redeem_script)
print(t.serialize().hex())

010000000126f6ae36ac4a1e2f41d6a3783a6e2a60a4bd5fd478c4873615e7831a5e28faae01000000da00483045022100a4e6fdb01de16c96aef524d44616cd7be78b4c61923dfbfb4f5fe9fde9dff6b80220160e9959c9a1345a5995d9ea4376be1275e21bce3f0301ef25870d4629b29281014730440220538cead804898256af9dc4d307013113971c02171823f396ea264b242395096c02207a214b8e2be2cc6ddce65463c7e8ad15db549d30fb1c19e911c913bd3bf188d7014752210223136797cb0d7596cb5bd476102fe3aface2a06338e1afabffacf8c3cab4883c210385c865e61e275ba6fda4a3167180fc5a6b607150ff18797ee44737cd0d34507b52aeffffffff01b76e8b000000000017a91436b865d5b9664193ea1db43d159edf9edf9438028700000000
